# SYNERGY LOGISTICS

## DESCRIPCIÓN DEL CASO
Synergy Logistics es una empresa dedicada a la intermediación de servicios de 
importación y exportación de diferentes productos. Actualmente la empresa 
cuenta con una base de datos que refleja las rutas más importantes que opera 
desde el año 2015, con su respectivo origen y destino, año, producto, modo de 
transporte y valor total. Su propósito, es que a partir de estos datos se genere un 
análisis que sirva de la base para la estructuración de su estrategia operativa.


## CONSIGNA
La Dirección de Synergy Logistics ha solicitado al equipo operativo, realizar una 
propuesta que permita enfocar las prioridades de la estrategia operativa 2021; para 
ello, se plantea analizar la viabilidad de 3 opciones de enfoque: rutas de importación 
y exportación, medio de transporte utilizado y valor total de importaciones y 
exportaciones.

##### Análisis de los Datos

Para comenzar importaremos las librerías que utilizaremos para el análisis de los datos

In [573]:
import pandas as pd 

Posteriormente importaremos los datos con los que vamos a trabajar

In [574]:
database = pd.read_csv("synergy_logistics_database.csv", index_col = 0, parse_dates=[5])

In [575]:
## Vista previa de los datos
database

,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
register_id,,,,,,,,,
1,Exports,Japan,China,2015,2015-01-31,Cars,Sea,Honda,33000000
2,Exports,Japan,China,2015,2015-01-02,Cars,Sea,Honda,16000000
3,Exports,Japan,China,2015,2015-02-02,Cars,Sea,Honda,29000000
4,Exports,Japan,China,2015,2015-03-02,Cars,Sea,Honda,14000000
5,Exports,Japan,China,2015,2015-04-02,Cars,Sea,Honda,17000000
...,...,...,...,...,...,...,...,...,...
19052,Imports,Japan,Singapore,2020,2020-06-27,Gas turbines,Sea,Union Energy Co,1000000
19053,Imports,Malaysia,Singapore,2020,2020-06-28,Gas turbines,Sea,Union Energy Co,2000000
19054,Imports,Malaysia,Singapore,2020,2020-06-29,Gas turbines,Sea,Union Energy Co,33000000


## Opción 1) Rutas de importación y exportación

Debido a los requerimientos de este análisis, lo más conveniente es filtrar los datos en dos categorias, las cuales serán Exportaciones e Importaciones, para lo cual se crean los DataFrames correspondientes:

In [576]:
exportaciones = database[database["direction"] == "Exports"]
importaciones = database[database["direction"] == "Imports"]

Una vez filtrados los datos, el primer análisis que se llevó a cabo es el de las rutas mayormente utilizadas para la actividad de exportaciones

In [577]:
top_exp = exportaciones.groupby(['origin','destination']).size()
top_exp = pd.DataFrame(top_exp).reset_index()
top_exp.columns.values[2] = "Exportaciones"
top_exp = top_exp.sort_values("Exportaciones", ascending=False).head(10)
top_exp = top_exp.rename(columns={"origin":"Origen", "destination":"Destino"})

#### 1.1) Rutas con mayor número de exportaciones 

In [578]:
top_exp

,Origen,Destino,Exportaciones
118,South Korea,Vietnam,497
95,Netherlands,Belgium,437
135,USA,Netherlands,436
32,China,Mexico,330
75,Japan,Brazil,306
51,Germany,France,299
115,South Korea,Japan,279
3,Australia,Singapore,273
24,Canada,Mexico,261
35,China,Spain,250


In [579]:
top_exp.iplot(kind='barh', x=['Origen', 'Destino'], y='Exportaciones', title="Número de Exportaciones", yTitle="Rutas")

Como podemos observar en la gráfica anterior, las 10 rutas más demandadas van desde las 250 exportaciones, hasta las 497, podríamos directamente implementar la estrategia de enfoque en ellas.

Análisis de exportaciones basado en el monto ingresado por ruta

In [580]:
top_val_exp = exportaciones.groupby(['origin','destination']).total_value.sum()
top_val_exp = pd.DataFrame(top_val_exp).reset_index()
top_val_exp.columns = ["Origen", "Destino", "Monto ($)"]
top_val_exp = top_val_exp.sort_values("Monto ($)", ascending=False).head(10)

#### 1.2) Rutas de exportación con mayor flujo de capital

In [581]:
top_val_exp 

,Origen,Destino,Monto ($)
32,China,Mexico,12250000000
24,Canada,Mexico,8450000000
118,South Korea,Vietnam,6877007000
38,France,Belgium,5538069000
47,France,United Kingdom,5427000000
34,China,South Korea,4790000000
134,USA,Mexico,4710000000
115,South Korea,Japan,4594000000
52,Germany,Italy,4541000000
30,China,Germany,4090000000


In [582]:
top_val_exp.iplot(kind='barh', x=["Origen", "Destino"], y="Monto ($)", title="Monto por Exportaciones", yTitle="Rutas")

En la gráfica anterior podemos observar ahora, las rutas con el monto mayor.

Como podemos observar, en ambas gráficas existen elementos en común, lo cual no indica que existen rutas que además de ser de las que más exportan, también son de las que mayor monto reflejan, por lo tanto, creo yo,  deberían ser las rutas a las que se implemente la estrategia que se tiene en mente.

A continuación se presentan dichas rutas:

In [583]:
conc_exp = top_exp.merge(top_val_exp, on=["Origen", "Destino"], how="left").dropna()
conc_exp

,Origen,Destino,Exportaciones,Monto ($)
0,South Korea,Vietnam,497,6.877007e+09
3,China,Mexico,330,1.225000e+10
6,South Korea,Japan,279,4.594000e+09
8,Canada,Mexico,261,8.450000e+09


Análisis de importaciones basadas en el número de veces que se realizó una ruta

In [584]:
top_imp = importaciones.groupby(['origin','destination']).size()
top_imp = pd.DataFrame(top_imp).reset_index()
top_imp.columns.values[2] = "Importaciones"
top_imp = top_imp.sort_values("Importaciones", ascending=False).head(10)
top_imp = top_imp.rename(columns={"origin":"Origen", "destination":"Destino"})

#### 1.3) Rutas con mayor número de Importaciones

In [585]:
top_imp

,Origen,Destino,Importaciones
33,Singapore,Thailand,273
12,Germany,China,233
4,China,Japan,210
20,Japan,Mexico,206
7,China,Thailand,200
26,Malaysia,Thailand,195
38,Spain,Germany,142
31,Mexico,USA,122
9,China,United Arab Emirates,114
1,Brazil,China,113


In [586]:
top_imp.iplot(kind='barh', x=["Origen", "Destino"], y="Importaciones", title="Número de Importaciones", yTitle="Rutas")

Análisis de importaciones basado en el monto ingresado por cada ruta

In [587]:
top_val_imp = importaciones.groupby(['origin','destination']).total_value.sum()
top_val_imp = pd.DataFrame(top_val_imp).reset_index()
top_val_imp.columns = ["Origen", "Destino", "Monto ($)"]
top_val_imp = top_val_imp.sort_values("Monto ($)", ascending=False).head(10)

#### 1.4) Rutas de importación con mayor flujo de capital

In [588]:
top_val_imp

,Origen,Destino,Monto ($)
33,Singapore,Thailand,4017000000
20,Japan,Mexico,3918000000
7,China,Thailand,3831000000
26,Malaysia,Thailand,3482000000
9,China,United Arab Emirates,3357000000
4,China,Japan,3237000000
31,Mexico,USA,2365000000
24,Japan,United Arab Emirates,2238000000
38,Spain,Germany,2044000000
14,Germany,Mexico,1804000000


In [589]:
top_val_imp.iplot(kind='barh', x=["Origen", "Destino"], y="Monto ($)", title="Monto por Importaciones", yTitle="Rutas")

Al igual que en el apartado de exportaciones, para poder observar las rutas que generan más importaciones y, a la vez, más flujo de capital, uniremos las dos tablas correspondientes dejando solamente rutas que se repitan en ambas.

In [590]:
conc_imp = top_imp.merge(top_val_imp, on=["Origen", "Destino"], how="left").dropna()

A continuación se muestran las rutas que predominan en ambos filtros y a las que deberian también implementarse las estrategias que se tienen pensadas:

In [591]:
conc_imp

,Origen,Destino,Importaciones,Monto ($)
0,Singapore,Thailand,273,4.017000e+09
2,China,Japan,210,3.237000e+09
3,Japan,Mexico,206,3.918000e+09
4,China,Thailand,200,3.831000e+09
5,Malaysia,Thailand,195,3.482000e+09
6,Spain,Germany,142,2.044000e+09
7,Mexico,USA,122,2.365000e+09
8,China,United Arab Emirates,114,3.357000e+09


### Conclusión:

Como podemos observar, se realizó el análisis de importaciones y exportaciones no sólo basado en el número de veces que se llevó a cabo cada ruta, si no también basados en el flujo de capital que se generó.
Lo que podemos observar es que las rutas no son exactamente las mismas en ambas situaciones, sin embargo cabe recalcar que hay rutas que se repiten y son a las que se debería poner más atención y dedicarle mayor esfuerzo por mantenerlas e incluso mejorarlas.

## Opción 2) Medio de Tranporte Utilizado

#### 2.1) Medios de transporte más utilizados para exportaciones

Para realizar este inciso, se agrupó el DataFrame "exportaciones", basado en la columna "transport_mode", la cual contiene los diversos medios de transporte en que se realizan los envíos. El código correspondiente quedó de la siguiente manera:

In [592]:
top_medio_exp = exportaciones.groupby("transport_mode").size()
top_medio_exp = pd.DataFrame(top_medio_exp).reset_index()
top_medio_exp.columns = ["Medio de Transporte", "Exportaciones"]
top_medio_exp = top_medio_exp.sort_values("Exportaciones", ascending=False)

In [593]:
top_medio_exp

,Medio de Transporte,Exportaciones
3,Sea,8434
1,Rail,2780
2,Road,2149
0,Air,2045


Se puede observar en la tabla que unicamente existen 4 medios de transporte, siendo el medio marino o las embarcaciones el más utilizado para realizar las exportaciones, siendo más de la mitad de ellas enviadas por este medio. A continuación se muestra la gráfica correspondiente:

In [594]:
top_medio_exp.iplot(kind='pie', labels="Medio de Transporte", values="Exportaciones")

#### 2.2) Medios de transporte con más ingresos para exportaciones

Además del número de exportaciones también se consideró el flujo de capital, el cual se obtuvo mediante el siguiente código:

In [595]:
medio_val_exp = exportaciones.groupby("transport_mode").total_value.sum()
medio_val_exp = pd.DataFrame(medio_val_exp).reset_index()
medio_val_exp.columns = ["Medio de Transporte", "Monto ($)"]
medio_val_exp = medio_val_exp.sort_values("Monto ($)", ascending=False)

In [596]:
medio_val_exp

,Medio de Transporte,Monto ($)
3,Sea,65592622000
1,Rail,34505043000
0,Air,32785147000
2,Road,27280486000


Podemos observar que los primeros 2 medios son los mismos que en la tabla correspondiente al número de exportaciones, y el único cambio que se hace es en el medio aereo, debido a que a pesar de tener menos exportaciones, tiene mayor flujo de capital. A continuación se muestra la gráfica:

In [597]:
medio_val_exp.iplot(kind='pie', labels="Medio de Transporte", values="Monto ($)")

#### 2.3) Medios de transporte más utilizados para importaciones

Aquí se realizó exactamente el mismo proceso que para las exportaciones, obteniendo los datos que se muestran a continuación:

In [598]:
top_medio_imp = importaciones.groupby("transport_mode").size()
top_medio_imp = pd.DataFrame(top_medio_imp).reset_index()
top_medio_imp.columns = ["Medio de Transporte", "Importaciones"]
top_medio_imp = top_medio_imp.sort_values("Importaciones", ascending=False)

In [599]:
top_medio_imp

,Medio de Transporte,Importaciones
3,Sea,2254
1,Rail,601
2,Road,449
0,Air,344


In [600]:
top_medio_imp.iplot(kind="pie", labels="Medio de Transporte", values="Importaciones")

#### 2.4) Medios de transporte con más ingresos para importaciones

In [601]:
medio_val_imp = importaciones.groupby("transport_mode").total_value.sum()
medio_val_imp = pd.DataFrame(medio_val_imp).reset_index()
medio_val_imp.columns = ["Medio de Transporte", "Monto ($)"]
medio_val_imp = medio_val_imp.sort_values("Monto ($)", ascending=False)

In [602]:
medio_val_imp

,Medio de Transporte,Monto ($)
3,Sea,34938000000
1,Rail,9123000000
2,Road,5990000000
0,Air,5477000000


In [603]:
medio_val_imp.iplot(kind="pie", labels="Medio de Transporte", values="Monto ($)")

### Conclusión:

Como podemos observar en los datos arrojados luego del análisis, tanto en importaciones como en exportaciones el medio de transporte más utilizado es el mismo (Mar). Tanto en número de veces, como en ingresos, el medio marino es, por mucho el preferido para realizar los envíos y en el que se debería prestar más empeño y dedicación debido a los buenos números que este arroja.

## Opción 3) Valor Total de Importaciones y Exportaciones

#### Valor Total de las Exportaciones

In [604]:
total_exp = exportaciones.total_value.sum()

In [605]:
print(f'El monto total por las exportaciones realizadas es de: $ {total_exp}')

El monto total por las exportaciones realizadas es de: $ 160163298000


#### Valor Total de las Importaciones

In [606]:
total_imp = importaciones.total_value.sum()


In [607]:
print(f'El monto total de las importaciones realizadas es de: $ {total_imp}')

El monto total de las importaciones realizadas es de: $ 55528000000


#### Diferencia entre Exportaciones e Importaciones

In [608]:
monto_total = total_exp - total_imp

In [609]:
print(f'Existe una diferencia de $ {monto_total } entre Exportaciones e Importaciones')

Existe una diferencia de $ 104635298000 entre Exportaciones e Importaciones


Podemos observar que existe una diferencia enorme entre las exportaciones y las importaciones, lo cual indica que los movimientos preferidos son de exportación. Y es el que más interés debería generar a la hora de ampliar el catálogo de paises o de rutas. 

Por último, obtendremos a los paises en los que Synergy Logistics debería enfocar sus esfuerzos, dichos paises que a continuación se mencionan representan casi el 80% del valor total de movimientos realizados (exportaciones e importaciones)

In [610]:
porc_exp = pd.DataFrame(exportaciones, columns=["origin", "total_value"])
porc_exp = porc_exp.groupby(["origin"]).total_value.sum()
porc_exp = pd.DataFrame(porc_exp).reset_index()
porc_exp = porc_exp.sort_values("total_value", ascending=False)
porc_exp["Porcentaje Acumulado"] = 100*(porc_exp.total_value.cumsum() / porc_exp.total_value.sum())
porc_exp.columns = ["País", "Monto ($)", "Porcentaje Acumulado"]
filtro = porc_exp["Porcentaje Acumulado"] <= 80
porc_exp_80 = porc_exp[filtro]

Estos son los paises que representan poco menos del 80% del valor de exportaciones realizadas y, por lo tanto, a los que mayor esfuerzo se les debería aplicar:

In [611]:
porc_exp_80

,País,Monto ($),Porcentaje Acumulado
5,China,32977046000,20.589640
6,France,18614332000,32.211736
18,USA,18355306000,43.672105
15,South Korea,14621146000,52.801004
13,Russia,13223000000,61.056953
10,Japan,11948976000,68.517449
7,Germany,11343233000,75.599741


In [612]:
porc_exp_80.iplot(kind="pie", labels="País", values=("Monto ($)"))

In [613]:
porc_imp = pd.DataFrame(importaciones, columns=["origin", "total_value"])
porc_imp = porc_imp.groupby(["origin"]).total_value.sum()
porc_imp = pd.DataFrame(porc_imp).reset_index()
porc_imp = porc_imp.sort_values("total_value", ascending=False)
porc_imp["Porcentaje Acumulado"] = 100*(porc_imp.total_value.cumsum() / porc_imp.total_value.sum())
porc_imp.columns = ["País", "Monto ($)", "Porcentaje Acumulado"]
filtro = porc_imp["Porcentaje Acumulado"] <= 80
porc_imp_80 = porc_imp[filtro]

Por otro lado, estos son los paises que representan poco menos del 80% del valor de importaciones realizadas y, por lo tanto, a los que mayor esfuerzo se les debería aplicar:

In [614]:
porc_imp_80

,País,Monto ($),Porcentaje Acumulado
3,China,12233000000,22.030327
7,Japan,8094000000,36.606757
14,USA,5291000000,46.135283
9,Mexico,4621000000,54.457211
5,Germany,4250000000,62.111007
11,Singapore,4017000000,69.345195
12,South Korea,3889000000,76.348869


In [615]:
porc_imp_80.iplot(kind="pie", labels="País", values="Monto ($)")

#### Analisis realizado por: Bibiano de la Cruz Romo
#### Para: Emtech Institute